# Term Project II

The main task is to build a personal food recipe recommendation web-application. The followings are its main features

- The user can search for recipe by name.
- The user can search for recipe by ingredients.
- The use can mark and unmark a particular recipe as her/his favourite recipe.
- The user receive the suggestion recipes she/he maybe interested in.

In [43]:
import pandas as pd

from sklearn import preprocessing
from sklearn import neighbors
from sklearn import tree
from sklearn import naive_bayes
from sklearn import model_selection
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from scipy import sparse

import numpy as np
from numpy import hstack

import multiprocessing as mp

import string

import json
import re

import nltk
from nltk import ngrams
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity

from spellchecker import SpellChecker
import os
from pathlib import Path

## Sample data

In [34]:
food = pd.read_csv('food_ingredients.csv')
food.drop('Unnamed: 0',axis='columns',inplace=True)
food.drop('Ingredients',axis='columns',inplace=True)
food.head()

,Title,Instructions,Image_Name,Cleaned_Ingredients
0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher..."
1,Crispy Salt and Pepper Potatoes,Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (..."
2,Thanksgiving Mac and Cheese,Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ..."
3,Italian Sausage and Bread Stuffing,Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in..."
4,Newton's Law,Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho..."


In [41]:
title = food.iloc[0]['Title']
print(title)

Miso-Butter Roast Chicken With Acorn Squash Panzanella


In [40]:
ingre = food.iloc[0]['Cleaned_Ingredients'].split(',')
ingre[0] = ingre[0].replace('[','')
ingre[-1] = ingre[-1].replace(']','')
for i in ingre :
    print(i)

'1 (3½–4-lb.) whole chicken'
 '2¾ tsp. kosher salt
 divided
 plus more'
 '2 small acorn squash (about 3 lb. total)'
 '2 Tbsp. finely chopped sage'
 '1 Tbsp. finely chopped rosemary'
 '6 Tbsp. unsalted butter
 melted
 plus 3 Tbsp. room temperature'
 '¼ tsp. ground allspice'
 'Pinch of crushed red pepper flakes'
 'Freshly ground black pepper'
 '⅓ loaf good-quality sturdy white bread
 torn into 1" pieces (about 2½ cups)'
 '2 medium apples (such as Gala or Pink Lady; about 14 oz. total)
 cored
 cut into 1" pieces'
 '2 Tbsp. extra-virgin olive oil'
 '½ small red onion
 thinly sliced'
 '3 Tbsp. apple cider vinegar'
 '1 Tbsp. white miso'
 '¼ cup all-purpose flour'
 '2 Tbsp. unsalted butter
 room temperature'
 '¼ cup dry white wine'
 '2 cups unsalted chicken broth'
 '2 tsp. white miso'
 'Kosher salt'
 'freshly ground pepper'


In [36]:
instru = ingre = food.iloc[0]['Instructions'].split('.')
for i in instru :
    print(i)

Pat chicken dry with paper towels, season all over with 2 tsp
 salt, and tie legs together with kitchen twine
 Let sit at room temperature 1 hour

Meanwhile, halve squash and scoop out seeds
 Run a vegetable peeler along ridges of squash halves to remove skin
 Cut each half into ½"-thick wedges; arrange on a rimmed baking sheet

Combine sage, rosemary, and 6 Tbsp
 melted butter in a large bowl; pour half of mixture over squash on baking sheet
 Sprinkle squash with allspice, red pepper flakes, and ½ tsp
 salt and season with black pepper; toss to coat

Add bread, apples, oil, and ¼ tsp
 salt to remaining herb butter in bowl; season with black pepper and toss to combine
 Set aside

Place onion and vinegar in a small bowl; season with salt and toss to coat
 Let sit, tossing occasionally, until ready to serve

Place a rack in middle and lower third of oven; preheat to 425°F
 Mix miso and 3 Tbsp
 room-temperature butter in a small bowl until smooth
 Pat chicken dry with paper towels, then r

In [42]:
image_name = food.iloc[0]['Image_Name']
print(image_name)

miso-butter-roast-chicken-acorn-squash-panzanella


![somehting](./food_image/miso-butter-roast-chicken-acorn-squash-panzanella.JPG)

# Spell Checker Feature

In [44]:
context = Path("eng-simple_wikipedia_2021_300K-sentences.txt").read_text("utf-8")
context = re.sub('[^A-Za-z]'," ",context)
context = " ".join(context.split())
context = context.lower()

list_of_word = context.split(" ")

In [47]:
spell = SpellChecker()

spell.word_frequency.load_words(list_of_word)

word = "Miso Buter Roats Chickken Wiht Acon Squah Panzanella"

misspelled = spell.unknown(word.split(" "))

if len(misspelled) != 0 :
    for word in misspelled:
        print(spell.correction(word))
        candidate_list = [i for i in spell.candidates(word)]
        print(candidate_list)
#         print(spell.candidates(word))
else:
    print("no mispelling")

chicken
['chicken']
squad
['squad', 'squab', 'shuah', 'squaw', 'squat', 'squash']
outer
['bute', 'buner', 'bunter', 'cuter', 'buler', 'buster', 'muter', 'guter', 'butter', 'butser', 'bater', 'butler', 'biter', 'buyer', 'puter', 'beter', 'outer']
roads
['rots', 'oats', 'roads', 'doats', 'coats', 'moats', 'rotas', 'roast', 'roasts', 'groats', 'boats', 'goats', 'roams', 'croats', 'routs', 'rats', 'roots', 'roat', 'roars']
manzanilla
['manzanilla']
con
['ason', 'con', 'akon', 'acun', 'lacon', 'aco', 'anon', 'bacon', 'amon', 'acdn', 'aron', 'acorn', 'macon', 'econ', 'icon', 'alon', 'aeon', 'axon', 'acou', 'avon', 'acton', 'aon', 'afon', 'apon']
